In [ ]:
import regex as re
import json
datafiles= {
  "E1" : [''],
  "E2" : ['a', 'b'],
  "E3" : [''],
  "E4" : ['']
}
datasets = []

for i,edition in enumerate(list(datafiles.keys())):
  dataset = ""
  for file in datafiles.get(edition):
    with open(f"./dataset/NF_{edition}{file}.txt", "r", encoding='utf-8') as fr:
      dataset += fr.read()
      fr.close()
  # for trash in re.findall(r"<h1.*>.*|<h2.*>.*|<h3.*>.*|<a.*>.*|<div.*>.*|<hr>", dataset):
  #   print(trash)
  datasets.append(re.sub(r"\n{2,}\s+(?=[a-z])", " ", re.sub(r"&.+;", "", dataset)))

In [ ]:
b_tag_dict = []
next_chars = 500
d = datasets[0]

# BUILD POSITIVE 
for match in re.finditer(r"((?<=<b>).+<\/b>)(.*(?<=<b>).+<\/b>)*", d):
  g1 = match.group(0)
  matched_b_tag = re.sub(r"</b>.*<b>|</b>"," ",g1).strip()
  end_of_b_tag = match.end()  
  surrounding_text_match = re.search(r"([^<]{1,"+str(next_chars)+r"})(?=<|$)", d[end_of_b_tag:end_of_b_tag+next_chars])
  surrounding_text = surrounding_text_match.group(0) if surrounding_text_match else ""

  short_def = re.sub(r"\s+", " ", surrounding_text).strip()
  if len(short_def) > 0:
    b_tag_dict.append([f"{matched_b_tag} {short_def}", matched_b_tag])


# BUILD NEGATIVE
for match in re.finditer(r"(\n\n[^<]{10,500})(?=\n|$|<)", d):
  g = match.group(0)
  matched_text = re.sub(r"\s+", " ", g).strip()
  b_tag_dict.append([matched_text, ""])

print(len(b_tag_dict))
with open("NF_E1_B.json", "w") as b_json:
  json.dump(b_tag_dict, b_json, indent=2, ensure_ascii=False)

In [ ]:
def find_sequential_page_patterns(text, remove=False):
    lines = text.splitlines()

    patterns_found = []
    line_indices_to_remove = set()


    for i in range(len(lines) - 2):
        try:
            first_num = int(lines[i])
            sec_num = int(lines[i + 4])
            if sec_num == first_num + 1:
                middle_text = lines[i + 2]
                pattern = f"{first_num}\n\n{middle_text}\n\n{sec_num}"
                patterns_found.append(pattern)
                line_indices_to_remove.update([i-1, i, i+1, i + 2, i + 3, i + 4, i + 5]) 
        except ValueError:
            continue

    print(f"Found {len(patterns_found)} patterns:\n")
    for i, pattern in enumerate(patterns_found, 1):
        print(f"Pattern #{i}:")
        print("-" * 40)
        print(pattern)
        print("-" * 40)
        print()
    #print (line_indices_to_remove)
    if remove:
        new_lines = [line for idx, line in enumerate(lines) if idx not in line_indices_to_remove]
        while new_lines and not new_lines[-1].strip():
            new_lines.pop()
        return '\n'.join(new_lines)
    return text
    #return patterns_found


def process_file(input_file_path, remove=False):
    try:
        with open(input_file_path, 'r', encoding='utf-8') as file:
            text = file.read()
        result = find_sequential_page_patterns(text, remove)

        if remove:
            with open("dataset/NF_E4.txt", 'w', encoding='utf-8') as file:
                file.write(result)
        return result
    except Exception as e:
        print(f"Error processing file: {str(e)}")
        return []

In [ ]:
#process_file("dataset/NF_E4.txt", remove=True)